In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

0.7819082736968994

In [3]:
casos = casos_confirmados.get_casos()
casos.shape

(329782, 15)

In [4]:
obitos = casos_confirmados.get_obitos()
obitos.shape

(6764, 11)

In [5]:
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
# duplicados_less = casos.loc[casos['hash_less'].isin(casos['hash'])].index.tolist()
# duplicados_more = casos.loc[casos['hash_more'].isin(casos['hash'])].index.tolist()
# dropar = set(duplicados + duplicados_less + duplicados_more)
# casos_dropar = casos.loc[dropar]
# casos_obitos_nao_dropar = casos_dropar.loc[casos_dropar['hash'].isin(obitos['hash'])].index.tolist()
# dropar = dropar - set(casos_obitos_nao_dropar)

casos.loc[duplicados,'motivo_exclusao'] = 'DUPLICIDADE'
casos.loc[duplicados,'hash'] = None
casos.loc[duplicados,'hash_less'] = None
casos.loc[duplicados,'hash_more'] = None

In [6]:
obitos = obitos.rename(columns={'data_do_obito': 'data_cura_obito'})
obitos['evolucao'] = 'OBITO'
obitos = obitos[['evolucao','data_cura_obito','hash']]

# casos.loc[casos['rs'].isnull(), 'mun_resid'] = casos.loc[casos['rs'].isnull(), 'mun_resid'] + '/' + casos.loc[casos['rs'].isnull(), 'uf_resid']

casos = pd.merge(left=casos, right=obitos, on='hash', how='left')

casos['rs'] = None
casos['is'] = None

casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','evolucao','data_cura_obito','motivo_exclusao']]
casos.shape

(329782, 18)

In [11]:
writer = pd.ExcelWriter("novos_casos_confirmados.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.to_excel(writer,index=None)

writer.save()
writer.close()